In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import os
import shutil

Extract the zip file:

In [5]:
zip_name = 'fruits-small.zip'
extracted_file = 'fruits-small'

# Make new directory:
os.makedirs(extracted_file, exist_ok = True)

try:
    shutil.unpack_archive(filename= zip_name, extract_dir= extracted_file)
    print('Successfully extacted the file')
except Exception as e:
    print('Error while unpacking the zip file.')

Successfully extacted the file


Steps for Model creation:
0. Data Preprocessing
    i. Image Generators
    ii. Splitting Train and Test data from Generators
1. Model architecting + Prepro
    i. CNN + ANN layers creation
2. Model Compilation
3. Model Training 
4. Model Evaluation
5. Model Testing / Deploy

In [ ]:
# Make data compatible using ImageDataGenerators:
# Each image in the Folders will be normalized using rescale.
train_image_generator= tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1.0/255.0)
test_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1.0/255.0)

In [7]:
# Splitting Test and Train Data:

train_image_data = train_image_generator.flow_from_directory('fruits-small/fruits-small/data/Training',
                                                             batch_size= 20,
                                                             class_mode = 'categorical',
                                                             target_size = (64,64))


test_image_data = test_image_generator.flow_from_directory('fruits-small/fruits-small/data/Validation',
                                                           batch_size= 20,
                                                           class_mode = 'categorical',
                                                           target_size= (64,64))

Found 3425 images belonging to 7 classes.
Found 1150 images belonging to 7 classes.


In [10]:
train_image_data.image_shape
test_image_data.image_shape

(64, 64, 3)

In [ ]:
# Model Architecting:
# CNN: A Convolution layer is combination of Convolve and Pooling:

model = tf.keras.Sequential()

# Conv2d(noofFeatureMaps, kernelShape, inputShape, activation) + padding = same -- add virtual boarder in every image
# NoOfFeatureMaps: Similar to no of units: Based on trial and error
# filter: (3,3), (4,4), (5,5) common :  in this content will be initialized randomly
# inputShape: similar to target_size

# ----------------- 1st convole layer:
model.add(tf.keras.layers.Conv2D(32, (3,3), input_shape= train_image_data.image_shape, activation='relu', padding= 'same'))
# Adding Pooling:
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

# ------------------- 2nd Convole layer:
model.add(tf.keras.layers.Conv2D(32, (3,3), input_shape= train_image_data.image_shape, activation='relu', padding= 'same'))
model.add(tf.keras.layers.MaxPool2D(pool_size = (2,2)))

# -------------------- Flatten:
model.add(tf.keras.layers.Flatten())

# FC Layer | ANN part, Layers creation:

model.add(tf.keras.layers.Dense(units= 100, activation='relu'))
model.add(tf.keras.layers.Dense(units= 100, activation='relu'))
model.add(tf.keras.layers.Dense(units= 256, activation= 'relu'))

# Output layer:
model.add(tf.keras.layers.Dense(units = 7, activation='softmax'))

# Model Summay:
model.summary()

AttributeError: module 'keras._tf_keras.keras.layers' has no attribute 'Conv2d'